¡Hola!<br />
Soy **Francisco Cortés**, estoy contento de revisar tu proyecto y ser parte de tu proceso de aprendizaje.
A lo largo del texto, haré algunas observaciones sobre mejoras en el código y también haré comentarios sobre tus percepciones sobre el tema. Si existe algún error en el código, no te preocupes, estoy aquí para ayudarte a mejorarlo, en la primera iteración te lo señalaré para que tengas la oportunidad de corregirlo, pero si aún no encuentras una solución para esta tarea, te daré una pista más precisa en la próxima iteración y también algunos ejemplos prácticos. Estaré abierto a retroalimentación y discusiones sobre el tema.<br />
Encontrarás mis comentarios a continuación - **por favor no los muevas, modifiques o borres**.
Revisaré cuidadosamente tu código para comprobar que se han cumplido con los requisitos y te proporcionaré mis comentarios en cajas verdes, amarillas o rojas como esta:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Si la ejecución fue perfecta succesfully.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Si existe alguna recomendación para que tu código mejore.
</div>

<div class="alert alert-block alert-danger">

<b>Comentario del revisor</b> <a class="tocSkip"></a>

Si existen correcciones necesarias para cumplir con los requisitos. El trabajo no puede ser aceptado si hay alguna caja roja.
</div>

Puedes responderme de la siguiente manera:

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
</div>

In [2]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine

# configuración de la base de datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

# cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# crear el motor de conexión
engine = create_engine(connection_string, connect_args={'sslmode': 'require', 'sslrootcert': 'CA.pem'})

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Correcto modo de crear la configuración inicial
</div>


# Objetivos del estudio.

- Determinar el número de libros publicados después del 1 de enero de 2000.

- Calcular el número de reseñas de usuarios y la calificación promedio para cada libro.

- Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.

- Encontrar al autor con la calificación promedio más alta para libros con al menos 50 calificaciones.

- Calcular el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

##  Número de libros publicados después del 1 de enero de 2000

In [3]:
num_libros_df = pd.read_sql_query("""
    SELECT COUNT(*) as num_libros
    FROM books
    WHERE publication_date > '2000-01-01';
""", engine)

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Correcto, buen uso del COUNT y WHERE
</div>


## Número de reseñas de usuarios y calificación promedio para cada libro.

In [4]:
resenas_calificacion_df = pd.read_sql_query("""
    SELECT b.title, COUNT(r.review_id) as num_resenas, AVG(rt.rating) as calificacion_promedio
    FROM books b
    LEFT JOIN reviews r ON b.book_id = r.book_id
    LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY b.title;
""", engine)

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Correcto, buena manera de unir las tablas y hacer el COUNT y el AVG
</div>


##  Editorial que ha publicado el mayor número de libros con más de 50 páginas.

In [5]:
editorial_mas_libros_df = pd.read_sql_query("""
    SELECT p.publisher, COUNT(b.book_id) as num_libros
    FROM publishers p
    JOIN books b ON p.publisher_id = b.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY num_libros DESC
    LIMIT 1;
""", engine)


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Correcto, Penguin Books es la editorial con más libros publicados, buena manera de encontrar y mostrar el resultado acertado
</div>


## Autor con la calificación promedio más alta para libros con al menos 50 calificaciones.

In [6]:
autor_mejor_calificado_df = pd.read_sql_query("""
    SELECT a.author, AVG(rt.rating) as calificacion_promedio
    FROM authors a
    JOIN books b ON a.author_id = b.author_id
    JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY a.author
    HAVING COUNT(rt.rating_id) >= 50
    ORDER BY calificacion_promedio DESC
    LIMIT 1;
""", engine)

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Bien hecho, buen uso de HAVING, los resultados son correctos
</div>


## Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [7]:
promedio_resenas_df = pd.read_sql_query("""
    WITH usuarios_con_mas_de_50_calificaciones AS (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50
    )
    SELECT AVG(num_resenas) as promedio_resenas
    FROM (
        SELECT COUNT(r.review_id) as num_resenas
        FROM reviews r
        JOIN usuarios_con_mas_de_50_calificaciones u ON r.username = u.username
        GROUP BY r.username
    ) as subquery;
""", engine)

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Bien hecho, buen uso del uso de subqueries, el resultado es acertado
</div>


In [8]:
# Imprimir los resultados
print("Número de libros publicados después del 1 de enero de 2000:")
print(num_libros_df)

print("\nNúmero de reseñas de usuarios y calificación promedio para cada libro:")
print(resenas_calificacion_df)

print("\nEditorial que ha publicado el mayor número de libros con más de 50 páginas:")
print(editorial_mas_libros_df)

print("\nAutor con la calificación promedio más alta para libros con al menos 50 calificaciones:")
print(autor_mejor_calificado_df)

print("\nNúmero promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros:")
print(promedio_resenas_df)

Número de libros publicados después del 1 de enero de 2000:
   num_libros
0         819

Número de reseñas de usuarios y calificación promedio para cada libro:
                                                 title  num_resenas  \
0                            The Count of Monte Cristo          115   
1                              Count Zero (Sprawl  #2)            4   
2    The Botany of Desire: A Plant's-Eye View of th...            4   
3                                 The Poisonwood Bible          110   
4                                 The Canterbury Tales           18   
..                                                 ...          ...   
994                           Of Love and Other Demons            4   
995  In the Heart of the Sea: The Tragedy of the Wh...            9   
996               Welcome to Temptation (Dempseys  #1)            4   
997                      World's End (The Sandman  #8)            4   
998                                  Holes (Holes  #1)     

## Conclusiones

- Número de libros publicados después del 1 de enero de 2000:

Se han publicado un total de 819 libros después del 1 de enero de 2000. Esto sugiere una producción literaria considerable en los últimos años, indicando un mercado activo para libros modernos.

- Número de reseñas de usuarios y calificación promedio para cada libro:

La tabla muestra una amplia gama de libros con diversas cantidades de reseñas y calificaciones promedio. Por ejemplo, "The Count of Monte Cristo" tiene 115 reseñas con una calificación promedio de 4.22, mientras que "Holes (Holes #1)" tiene 155 reseñas con una calificación promedio de 3.97.

Estos datos indican que hay libros muy populares con un alto número de reseñas, así como libros con menos reseñas pero con calificaciones altas, lo que puede indicar una alta calidad percibida por un grupo más pequeño de lectores.

- Editorial que ha publicado el mayor número de libros con más de 50 páginas:

Penguin Books ha publicado la mayor cantidad de libros con más de 50 páginas, con un total de 42 libros. Esto sugiere que Penguin Books es una editorial prominente y activa, conocida por su amplia gama de publicaciones de calidad.

- Autor con la calificación promedio más alta para libros con al menos 50 calificaciones:

Diana Gabaldon es la autora con la calificación promedio más alta (4.3) entre aquellos libros que han recibido al menos 50 calificaciones. Esto indica una alta satisfacción entre los lectores de sus libros, lo que sugiere que ella tiene una base de lectores leales y una reputación de alta calidad en su obra.

- Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros:

Los usuarios que han calificado más de 50 libros tienen un promedio de 24.33 reseñas de texto. Este dato es interesante porque muestra que los usuarios más activos en términos de calificaciones también contribuyen significativamente con reseñas detalladas, lo que podría ser valioso para otros lectores que buscan opiniones detalladas antes de decidir qué libro leer.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br />
Has realizado un buen trabajo, me doy cuenta de que has aplicado los conocimientos que has adquirido durante el curso y los procedimientos realizados son correctos, SQL es una gran herramienta que nos ayuda en muchos casos y es bastante útil en proyectos reales.<br/>
    
Me parece interesante el modo en el que has resuelto el ejercicio, al separar primero los queries y después mostrarlos todos en una sola celda, al estar todos juntos recomendaria limitar la salida de los datos (LIMIT 5) para que puedan ser leidos más facilmente, de cualquier manera me parece muy ordenada la forma en la que lo has realizado.<br/>

Continúa con el buen trabajo y mucho éxito en el siguiente Sprint!
</div>
